# Exercise 2: Feature Engineering (5 Points)

##### In the last exercise we found out that the zipCode alone is not improving the classification a lot. Hence, we want to add another feature to finally get better predictions:

##### Have a look at the `apply` function on Pandas DataFrames (documentation). You can use this function to craft new features out of the `basket` column: For every category in the basket (there are only 6: from 0 to 5), craft a feature that counts how often the category appears in the basket. Train a logistic regression now also including these new features and check the accuracy on the test data.

##### See the attached screenshot for an example.

##### Hint: After adding the new features, you have to drop the original `basket` column before training (the classifier would not train on a `list` type anyway).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json

transactions = []
days_in_month = 31
for i in range (1,days_in_month+1):
  for line in open('data/return-data/2017-01-'+str(i).zfill(2)+'.txt', 'r'):
    transactions.append(json.loads(line))
transactions_df = pd.DataFrame.from_dict(transactions)
transactions_df.head()

,basket,returnLabel,totalAmount,transactionId,zipCode
0,"[4, 3, 0, 0, 4, 4, 0, 4, 2, 5, 4]",0,693,8220990793,2233
1,"[3, 1, 2, 2, 1, 0, 1]",0,427,9895968997,1854
2,[5],0,59,7013881318,7480
3,"[0, 4]",0,86,2976689328,1166
4,"[3, 1, 2, 3, 3, 5]",0,312,9014690264,6574


In [2]:
transactions_df = transactions_df.drop('transactionId', axis=1)
transactions_df = transactions_df.drop('zipCode', axis=1)
transactions_df.head()

,basket,returnLabel,totalAmount
0,"[4, 3, 0, 0, 4, 4, 0, 4, 2, 5, 4]",0,693
1,"[3, 1, 2, 2, 1, 0, 1]",0,427
2,[5],0,59
3,"[0, 4]",0,86
4,"[3, 1, 2, 3, 3, 5]",0,312


In [3]:
transactions_df["count_0"] = 7 # placeholder
transactions_df["count_1"] = 7
transactions_df["count_2"] = 7
transactions_df["count_3"] = 7
transactions_df["count_4"] = 7
transactions_df["count_5"] = 7

transactions_df.head()

,basket,returnLabel,totalAmount,count_0,count_1,count_2,count_3,count_4,count_5
0,"[4, 3, 0, 0, 4, 4, 0, 4, 2, 5, 4]",0,693,7,7,7,7,7,7
1,"[3, 1, 2, 2, 1, 0, 1]",0,427,7,7,7,7,7,7
2,[5],0,59,7,7,7,7,7,7
3,"[0, 4]",0,86,7,7,7,7,7,7
4,"[3, 1, 2, 3, 3, 5]",0,312,7,7,7,7,7,7


In [4]:
for i in transactions_df.index:
    transactions_df["count_0"].at[i] = transactions_df.basket[i].count(0)
    transactions_df["count_1"].at[i] = transactions_df.basket[i].count(1)
    transactions_df["count_2"].at[i] = transactions_df.basket[i].count(2)
    transactions_df["count_3"].at[i] = transactions_df.basket[i].count(3)
    transactions_df["count_4"].at[i] = transactions_df.basket[i].count(4)
    transactions_df["count_5"].at[i] = transactions_df.basket[i].count(5)
transactions_df.head()

,basket,returnLabel,totalAmount,count_0,count_1,count_2,count_3,count_4,count_5
0,"[4, 3, 0, 0, 4, 4, 0, 4, 2, 5, 4]",0,693,3,0,1,1,5,1
1,"[3, 1, 2, 2, 1, 0, 1]",0,427,1,3,2,1,0,0
2,[5],0,59,0,0,0,0,0,1
3,"[0, 4]",0,86,1,0,0,0,1,0
4,"[3, 1, 2, 3, 3, 5]",0,312,0,1,1,3,0,1


In [5]:
transactions_df = transactions_df.drop('basket', axis=1)
transactions_df.head()

,returnLabel,totalAmount,count_0,count_1,count_2,count_3,count_4,count_5
0,0,693,3,0,1,1,5,1
1,0,427,1,3,2,1,0,0
2,0,59,0,0,0,0,0,1
3,0,86,1,0,0,0,1,0
4,0,312,0,1,1,3,0,1


In [6]:
from sklearn.model_selection import train_test_split

X = transactions_df.drop('returnLabel',axis=1)
y = transactions_df.returnLabel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [7]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(solver="liblinear")
logmodel.fit(X_train, y_train)

predictions = logmodel.predict(X_test)

In [8]:
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

print(classification_report(y_test,predictions))
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      9047
           1       0.66      0.28      0.40       253

   micro avg       0.98      0.98      0.98      9300
   macro avg       0.82      0.64      0.69      9300
weighted avg       0.97      0.98      0.97      9300

Accuracy: 0.9765591397849462
